## 1. Import packages

In [1]:
# Import packages
import hail as hl
from bokeh.io import output_notebook,show
import gnomad.utils.vep

## 2. Import data

In [2]:
# Import gnomaAD v.3.1.2
ht = hl.read_table('gs://gcp-public-data--gnomad/release/3.1.2/ht/genomes/gnomad.genomes.v3.1.2.sites.ht')
ht = ht.head(100000) # Subset the data

# Import mutation rates from gnomAD paper
ht_mu = hl.import_table('data/supplementary_dataset_10_mutation_rates.tsv.gz',
                delimiter='\t', impute=True, force_bgz=True)
ht_mu = ht_mu.key_by('context', 'ref', 'alt') # has to have a key in order to join using foreign key

# Import context table from gnomad (https://broadinstitute.github.io/gnomad_methods/api_reference/utils/vep.html?highlight=context#gnomad.utils.vep.get_vep_context)
context_table = gnomad.utils.vep.get_vep_context("GRCh38").ht()
context_table_parsed = context_table.select(context_table.context)
context_table_parsed = context_table_parsed.transmute(context = context_table_parsed.context[2:5])

Initializing Hail with default parameters...


2022-09-26 18:37:05 WARN  Utils:69 - Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.181 instead (on interface en0)
2022-09-26 18:37:05 WARN  Utils:69 - Set SPARK_LOCAL_IP if you need to bind to another address


2022-09-26 18:37:06 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://192.168.0.181:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.99-57537fea08d4
LOGGING: writing to /Users/adrian/BroadIS/01_maps/hail-20220926-1837-0.2.99-57537fea08d4.log
2022-09-26 18:37:20 Hail: INFO: Reading table to impute column types
2022-09-26 18:37:23 Hail: INFO: Finished type imputation
  Loading field 'context' as type str (imputed)
  Loading field 'ref' as type str (imputed)
  Loading field 'alt' as type str (imputed)
  Loading field 'methylation_level' as type int32 (imputed)
  Loading field 'mu_snp' as type float64 (imputed)


### Show the data structure

In [3]:
# No methylation field present 
ht.describe()

----------------------------------------
Global fields:
    'freq_meta': array<dict<str, str>> 
    'freq_index_dict': dict<str, int32> 
    'faf_index_dict': dict<str, int32> 
    'faf_meta': array<dict<str, str>> 
    'vep_version': str 
    'vep_csq_header': str 
    'dbsnp_version': str 
    'filtering_model': struct {
        model_name: str, 
        score_name: str, 
        snv_cutoff: struct {
            bin: float64, 
            min_score: float64
        }, 
        indel_cutoff: struct {
            bin: float64, 
            min_score: float64
        }, 
        model_id: str, 
        snv_training_variables: array<str>, 
        indel_training_variables: array<str>
    } 
    'age_distribution': struct {
        bin_edges: array<float64>, 
        bin_freq: array<int32>, 
        n_smaller: int32, 
        n_larger: int32
    } 
    'freq_sample_count': array<int32> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<s

In [4]:
ht.show(3)

+---------------+------------+
| locus         | alleles    |
+---------------+------------+
| locus<GRCh38> | array<str> |
+---------------+------------+
| chr1:10031    | ["T","C"]  |
| chr1:10037    | ["T","C"]  |
| chr1:10043    | ["T","C"]  |
+---------------+------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,56642,0),(2,1.65e-05,121094,0),(0,0.00e+00,25546,0),(0,0.00e... |
| [(2,2.60e-05,76882,0),(4,3.15e-05,126902,0),(0,0.00e+00,34670,0),(1,1.80e... |
| [(1,1.17e-05,85634,0),(1,8.24e-06,121430,0),(0,0.00e+00,39236,0),(0,0.00e... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.gq_hist_all.bin_freq                                          |
+------------------------------------------------------------------------------+
| array<int64>                                                                 |
+------------------------------------------------------------------------------+
| [14149,7330,6840,3904,23217,2436,1647,594,176,118,49,29,19,9,11,6,5,2,4,2]   |
| [10121,5441,5610,3838,29394,3741,2868,1281,401,358,163,63,69,31,18,20,10,... |
| [6578,3782,4202,3336,31093,4532,3696,1752,595,585,261,88,82,59,17,26,14,5... |
+------------------------------------------------------------------------------+

+--------------------------------------+-------------------------------------+
| raw_qual_hists.gq_hist_all.n_smaller | raw_qual_hists.gq_hist_all.n_larger |
+--------------------------------------+-------------------------------------+
|                                int64 |                               int64 |
+--------------------------------------+-------------------------------------+
|                                    0 |                                   0 |
|                                    0 |                                   0 |
|                                    0 |                                   0 |
+--------------------------------------+-------------------------------------+

+------------------------------------------------------------------------------+
| raw_qual_hists.dp_hist_all.bin_edges                                         |
+------------------------------------------------------------------------------+
| array<float64>                                                               |
+------------------------------------------------------------------------------+
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
| [0.00e+00,5.00e+00,1.00e+01,1.50e+01,2.00e+01,2.50e+01,3.00e+01,3.50e+01,... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| raw_q

In [5]:
# Table with methylation level and mutational rate in the trinucleotide context
ht_mu.show(3)

,,,,
context,ref,alt,methylation_level,mu_snp
str,str,str,int32,float64
"""AAA""","""A""","""C""",0,1.83e-09
"""AAA""","""A""","""G""",0,3.03e-09
"""AAA""","""A""","""T""",0,1.07e-09


In [6]:
# This table contains already precalculated nucleotides -3/+3 from mutation site 
context_table_parsed.show(3)

,,
locus,alleles,context
locus<GRCh38>,array<str>,str
chr1:10001,"[""T"",""A""]","""NTA"""
chr1:10001,"[""T"",""C""]","""NTA"""
chr1:10001,"[""T"",""G""]","""NTA"""


## 3. Add context field to main data

In [7]:
# Before joining the tri-nucleotide context of mutation
ht.count()

100000

In [8]:
# Join only matching rows from context to ht table.
#ht = ht.key_by('locus', 'alleles').join(context_table_parsed.key_by('locus', 'alleles'), how = 'left') # resorts the data making it slow
ht = ht.annotate(**context_table_parsed[ht.locus, ht.alleles])


In [9]:
# After
ht.count()

100000

## 4. Add mutation rates for added contexts

In [10]:
# Split alleles field to ref and alt allele
ht = ht.annotate(ref=ht.alleles[0], alt=ht.alleles[1])

# Add mutation rates according to the context, but also ref and alt allele for this context
#ht = ht.key_by("context", "ref", "alt").join(ht_mu.key_by("context", "ref", "alt"), how = 'left') # resorts the data making it slow
ht = ht.annotate(**ht_mu[ht.context, ht.ref, ht.alt])


In [11]:
# After adding context and mutation rates to the main table 
# (can be more than original number of rows as context may occure more than once depending on the locus)
ht.count()

100000

In [12]:
# Show that contexts may be the same, but locus is completely different
ht.show(20)

2022-09-26 18:39:40 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-26 18:39:41 Hail: INFO: Coerced sorted dataset
2022-09-26 18:45:52 Hail: INFO: Ordering unsorted dataset with network shuffle4]


+---------------+
| locus         |
+---------------+
| locus<GRCh38> |
+---------------+
| chr1:10031    |
| chr1:10037    |
| chr1:10043    |
| chr1:10055    |
| chr1:10057    |
| chr1:10061    |
| chr1:10061    |
| chr1:10064    |
| chr1:10067    |
| chr1:10108    |
| chr1:10108    |
| chr1:10108    |
| chr1:10109    |
| chr1:10109    |
| chr1:10111    |
| chr1:10113    |
| chr1:10114    |
| chr1:10114    |
| chr1:10114    |
| chr1:10114    |
+---------------+

+------------------------------------------------------------------------------+
| alleles                                                                      |
+------------------------------------------------------------------------------+
| array<str>                                                                   |
+------------------------------------------------------------------------------+
| ["T","C"]                                                                    |
| ["T","C"]                                                                    |
| ["T","C"]                                                                    |
| ["T","C"]                                                                    |
| ["A","C"]                                                                    |
| ["T","C"]                                                                    |
| ["T","TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC"]                     |
| ["C","CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCAACCCTAACCCTAACCCTAACC... |
| ["T","TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC"]                           |
| ["C","CA"]                                                                   |
| ["C","CAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT"]                          |
| ["CAACCCT","C"]                                                              |
| ["A","T"]                                                                    |
| ["AACCCT","A"]                                                               |
| ["C","A"]                                                                    |
| ["CT","C"]                                                                   |
| ["T","A"]                                                                    |
| ["T","C"]                                                                    |
| ["TA","T"]                                                                   |
| ["TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCTAACC... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(0,0.00e+00,56642,0),(2,1.65e-05,121094,0),(0,0.00e+00,25546,0),(0,0.00e... |
| [(2,2.60e-05,76882,0),(4,3.15e-05,126902,0),(0,0.00e+00,34670,0),(1,1.80e... |
| [(1,1.17e-05,85634,0),(1,8.24e-06,121430,0),(0,0.00e+00,39236,0),(0,0.00e... |
| [(1,1.06e-05,94152,0),(5,4.65e-05,107630,0),(0,0.00e+00,44378,0),(1,1.80e... |
| [(3,2.64e-05,113536,0),(3,2.41e-05,124252,0),(2,3.78e-05,52912,0),(0,0.00... |
| [(0,0.00e+00,108768,0),(2,1.70e-05,117852,0),(0,0.00e+00,50916,0),(0,0.00... |
| [(0,0.00e+00,108768,0),(7,5.94e-05,117852,0),(0,0.00e+00,50916,0),(0,0.00... |
| [(0,0.00e+00,149580,0),(1,6.62e-06,150996,0),(0,0.00e+00,67168,0),(0,0.00... |
| [(2,1.67e-05,119758,0),(10,7.87e-05,127024,1),(0,0.00e+00,55000,0),(2,2.7... |
| [(1,9.91e-05,10090,0),(1,7.19e-05,13906,0),(1,2.16e-04,4632,0),(0,0.00e+0... |
| [(2,1.98e-04,10090,0),(2,1.44e-04,13906,0),(2,4.32e-04,4632,0),(0,0.00e+0... |
| [(0,0.00e+00,10088,0),(5,3.60e-04,13906,0),(0,0.00e+00,4632,0),(0,0.00e+0... |
| [(0,0.00e+00,1556,0),(1,1.51e-04,6624,0),(0,0

## 5. Train linear model on synonymous variants for mutational class correction

In [52]:
def train_on_synonymous(ht):
    """
    This function trains the regression model to predict the expected singleton/total variants ratio.
    
    Requirements (things to change as for best parcitces)
    1. Already annotated mutation rates for each tri-nucleotide context in the main hail table
    """
    # Filter only synonymous variants
    ht_syn = ht.filter(ht.vep.most_severe_consequence == "synonymous_variant")

    # Calculate number of variants in each tri-nucleotide context in synonymous variants
    ht_syn_N_variants = (ht_syn.group_by(ht_syn.context, ht_syn.ref, ht_syn.alt, ht_syn.mu_snp).aggregate(N_variants = hl.agg.count()))

    # Calculate number of singletons for each tri-nucleotide context in synonymous variants
    ht_syn_singletons = ht_syn.filter(ht_syn.info.singleton == 1)
    ht_syn_N_singletons = (ht_syn_singletons.group_by(ht_syn_singletons.context, ht_syn_singletons.ref, ht_syn_singletons.alt, ht_syn_singletons.mu_snp).aggregate(N_singletons = hl.agg.count()))

    # Merge the N variants and N singletons tables
    ht_syn_ps  = ht_syn_N_variants.join(ht_syn_N_singletons, how = 'outer') # outer as all will match and we want all
    ht_syn_ps = ht_syn_ps.annotate(ps = ht_syn_ps.N_singletons/ht_syn_ps.N_variants)
    
    return ht_syn_ps 

In [53]:
ht_syn_ps = train_on_synonymous(ht)

2022-09-26 19:20:28 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'filtering_model' -> 'filtering_model_1'
    'freq_meta' -> 'freq_meta_1'
    'freq_sample_count' -> 'freq_sample_count_1'
    'freq_index_dict' -> 'freq_index_dict_1'
    'vep_version' -> 'vep_version_1'
    'dbsnp_version' -> 'dbsnp_version_1'
    'faf_meta' -> 'faf_meta_1'
    'vep_csq_header' -> 'vep_csq_header_1'
    'age_distribution' -> 'age_distribution_1'
    'faf_index_dict' -> 'faf_index_dict_1'


In [54]:
ht_syn_ps.show(3)

2022-09-26 19:24:03 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-26 19:24:03 Hail: INFO: Coerced sorted dataset
2022-09-26 19:26:42 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]
2022-09-26 19:31:25 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-26 19:31:26 Hail: INFO: Coerced sorted dataset
2022-09-26 19:34:31 Hail: INFO: Coerced sorted dataset             (8 + 6) / 14]


,,,,,,
context,ref,alt,mu_snp,N_variants,N_singletons,ps
str,str,str,float64,int64,int64,float64
"""AAT""","""A""","""G""",7.75e-09,1,1,1.00e+00
"""ACA""","""C""","""T""",9.32e-09,2,NA,NA
"""ACC""","""C""","""A""",5.70e-09,1,NA,NA


### Show input table for regression

In [16]:
# How many rows after adding mutation rates
ht_syn_ps.count()

2022-09-23 00:37:21 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:37:22 Hail: INFO: Coerced sorted dataset
2022-09-23 00:38:02 Hail: INFO: Coerced sorted dataset============(14 + 0) / 14]
2022-09-23 00:38:48 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-23 00:38:48 Hail: INFO: Coerced sorted dataset
2022-09-23 00:39:29 Hail: INFO: Coerced sorted dataset====>       (12 + 2) / 14]


69

### Perform regression

In [55]:
# Perform regression
ht_syn_lm = ht_syn_ps.aggregate(hl.agg.linreg(ht_syn_ps.ps, [1, ht_syn_ps.mu_snp], weight=ht_syn_ps.N_variants).beta)

# Show intercept and beta
ht_syn_lm

2022-09-26 19:38:42 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-26 19:38:43 Hail: INFO: Coerced sorted dataset
2022-09-26 19:41:09 Hail: INFO: Coerced sorted dataset             (0 + 8) / 14]
2022-09-26 19:42:01 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-26 19:42:02 Hail: INFO: Coerced sorted dataset
2022-09-26 19:43:10 Hail: INFO: Coerced sorted dataset============(14 + 0) / 14]


[0.664961864082555, 356649.8405147907]

## 6. Predict expected number of variants for each context

### For testing purposes focus on `3_prime_UTR_variant`

### Function for regression eventually will be made starting here and put in `/utils/utils.py` script

In [72]:
## Define regression function
def regress_per_context(ht, ht_syn_lm):
    """
    Calculates MAPS with Standard Error of the Mean for a given consequence.
    
    Requires input in form of Hail table: (so the things that should be changes for easier accessibility):
        1) Mutation rates are to be annotated for each tri-nucleotide context. 
    """
    ht_reg_table = ht.annotate(consequence = ht.vep.most_severe_consequence)

    # Count number of variants and singletons
    ht_reg_table_N_variants = (ht_reg_table.group_by(ht_reg_table.context, ht_reg_table.ref, ht_reg_table.alt, ht_reg_table.mu_snp, ht_reg_table.consequence).aggregate(N_variants = hl.agg.count()))
    ht_reg_table_N_singletons = (ht_reg_table.group_by(ht_reg_table.context, ht_reg_table.ref, ht_reg_table.alt, ht_reg_table.mu_snp, ht_reg_table.consequence).aggregate(N_singletons = hl.agg.sum(ht_reg_table.info.singleton)))

    # Merge the tables to obtain proportions (ps)
    ht_reg_table_ps = ht_reg_table_N_variants.annotate(**ht_reg_table_N_singletons[ht_reg_table_N_variants.context, ht_reg_table_N_variants.ref, ht_reg_table_N_variants.alt, ht_reg_table_N_variants.mu_snp, ht_reg_table_N_variants.consequence])
    ht_reg_table_ps = ht_reg_table_ps.annotate(ps = ht_reg_table_ps.N_singletons/ht_reg_table_ps.N_variants)
    
    # Get expected number of singletons by applying the model factors
    ht_reg_table_ps_lm_cons = ht_reg_table_ps.annotate(expected_singletons=(ht_reg_table_ps.mu_snp * ht_syn_lm[1] + ht_syn_lm[0]) * ht_reg_table_ps.N_variants)

    # To aggregate just sum for the context
    ht_reg_table_ps_lm_cons_agg = (ht_reg_table_ps_lm_cons.group_by("consequence")
                  .aggregate(N_singletons=hl.agg.sum(ht_reg_table_ps_lm_cons.N_singletons),
                             expected_singletons=hl.agg.sum(ht_reg_table_ps_lm_cons.expected_singletons),
                             N_variants=hl.agg.sum(ht_reg_table_ps_lm_cons.N_variants)))

    # Calculate MAPS and aggregated proportions 
    ht_reg_table_ps_lm_cons_agg_MAPS = ht_reg_table_ps_lm_cons_agg.annotate(ps_agg=ht_reg_table_ps_lm_cons_agg.N_singletons / ht_reg_table_ps_lm_cons_agg.N_variants,
        maps=(ht_reg_table_ps_lm_cons_agg.N_singletons - ht_reg_table_ps_lm_cons_agg.expected_singletons) / ht_reg_table_ps_lm_cons_agg.N_variants)

    # Add MAPS standard error of the mean (sem)
    ht_reg_table_ps_lm_cons_agg_MAPS = ht_reg_table_ps_lm_cons_agg_MAPS.annotate(maps_sem=(ht_reg_table_ps_lm_cons_agg_MAPS.ps_agg * (1 - ht_reg_table_ps_lm_cons_agg_MAPS.ps_agg) / ht_reg_table_ps_lm_cons_agg_MAPS.N_variants) ** 0.5)
    return ht_reg_table_ps_lm_cons_agg_MAPS


In [ ]:
regress_per_context(ht, ht_syn_lm).show(100)

2022-09-26 20:14:39 Hail: INFO: Ordering unsorted dataset with network shuffle4]
2022-09-26 20:14:40 Hail: INFO: Coerced sorted dataset
